### Examples of how to use `torch.optim.SGD` 
 + The Rosenbrock function is minimized
 + In a similar way `torch.optim.Adam` could be used

In [ ]:
%matplotlib inline

import sys
import numpy as np
import scipy.optimize as so

sys.path.append('..')
from utils import rosenbrock_contour, rosenbrock, gd

### Set initial condition

In [ ]:
x0 = np.array([-1.9, 2])

### Powell method (just for fun)

In [ ]:
rosenbrock_contour(so.fmin_powell(rosenbrock, x0=x0, retall=True)[1])

### Gradient descent

In [ ]:
lrs = [1e-3/2, 3*1e-3/2, 2*1e-3]  # learning rates to test with

rosenbrock_contour({round(lr, 4): gd(x0, lr) for lr in lrs})
rosenbrock_contour({round(lr, 4): gd(x0, lr, use_torch_opt=True) for lr in lrs})

### Gradient descent with momentum

In [ ]:
rosenbrock_contour(**gd(x0, lr=1e-3/2, mu=0.9))
rosenbrock_contour(**gd(x0, lr=1e-3/2, mu=0.9, use_torch_opt=True))

### Nesterov accelerated gradient

In [ ]:
rosenbrock_contour(**gd(x0, lr=1e-3/2, mu=0.9, nesterov=True))
rosenbrock_contour(**gd(x0, lr=1e-3/2, mu=0.9, nesterov=True, use_torch_opt=True))

### Nesterov accelerated gradient with learning rate scheduler

In [ ]:
rosenbrock_contour(**gd(x0, lr=1e-3/2, mu=0.9, nesterov=True, schedule_lr=True))